In [1]:
import os

In [2]:
%pwd

'd:\\Runway\\Projects\\data_it_enterprises\\research'

In [3]:
os.chdir("../")

In [12]:
%pwd

'd:\\Runway\\Projects\\data_it_enterprises'

In [19]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    #alpha: float
    #l1_ratio: float
    target_column: float
    objective: str
    colsample_bytree: float
    learning_rate: float
    max_depth: int 
    alpha: int
    n_estimators: int



In [20]:
from data_it_entp.constants import *
from data_it_entp.utils.common import read_yaml,create_directories

In [21]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.Xgboost
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            #alpha = params.alpha,
            #l1_ratio = params.l1_ratio,
            target_column = schema.name,
            objective= params.objective,
            colsample_bytree =params.colsample_bytree,
            learning_rate= params.learning_rate,
            max_depth= params.max_depth,
            alpha = params.alpha,
            n_estimators = params.n_estimators            
        )

        return model_trainer_config

In [22]:
import pandas as pd
import os
from data_it_entp import logger
from sklearn.linear_model import ElasticNet
import xgboost as xgb
import joblib

In [23]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        features = ['CPU_Load', 'Memory_Usage', 'Disk_Usage']


        train_x = train_data[features]
        print(f"train x={train_x}")
        test_x = test_data[features]
        print(f"test x={test_x}")
        train_y = train_data[[self.config.target_column]]
        print(train_y)
        test_y = test_data[[self.config.target_column]]
        print(test_y)


        #lr = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)

        xg_reg = xgb.XGBRegressor(objective = self.config.objective , colsample_bytree = self.config.colsample_bytree, learning_rate = self.config.learning_rate,
                          max_depth = self.config.max_depth, alpha = self.config.alpha, n_estimators = self.config.n_estimators)

        #lr.fit(train_x, train_y)
        xg_reg.fit(train_x, train_y)


        #joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))
        joblib.dump(xg_reg, os.path.join(self.config.root_dir, self.config.model_name))

In [11]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-01-26 12:37:11,686:INFO:common:yaml file: config\config.yaml loaded successfully:]
[2024-01-26 12:37:11,737:INFO:common:yaml file: params.yaml loaded successfully:]
[2024-01-26 12:37:11,759:INFO:common:yaml file: schema.yaml loaded successfully:]
[2024-01-26 12:37:11,763:INFO:common:created directory at: artifacts:]
[2024-01-26 12:37:11,766:INFO:common:created directory at: artifacts/model_trainer:]


train x=       CPU_Load  Memory_Usage  Disk_Usage
0     47.170597     55.417126   49.655354
1     51.224043     48.644386   29.545440
2     62.344658     40.880425   65.341062
3     30.592252     76.811635   44.002248
4     31.403968     64.144694   35.424983
...         ...           ...         ...
3745  33.485037     54.120987   52.408981
3746  41.602582     28.169491   59.457946
3747  58.798479     37.142573   52.071868
3748  53.980931     58.870000   56.398904
3749  52.509694     48.583583   44.554877

[3750 rows x 3 columns]
test x=       CPU_Load  Memory_Usage  Disk_Usage
0     26.224957     34.397458   39.018992
1     55.979687     38.774207   56.923488
2     59.279552     26.559363   56.570760
3     59.978793     46.729371   43.289534
4     71.310066      5.786738   61.856858
...         ...           ...         ...
1245  44.765197     46.087772   43.211689
1246  45.175807     27.807073   24.542793
1247  64.178300     37.871120   66.206010
1248  50.834585     40.128565   56.9